In [119]:
import torch
from torch import nn
from d2l import torch as d2l

In [120]:
def compute_conv2d(conv2d, X):
    # (1, 1) will help modify the shape of X into a suitable shape
    # for conv2d computation.
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    # Just like the (1, 1) above, this method remove the (1, 1) in
    # return results
    #print("Y is: ", Y)
    return Y.reshape(Y.shape[2:])

conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1)
X = torch.rand((8, 8))
compute_conv2d(conv2d, X).shape

torch.Size([8, 8])

In [121]:
conv2d = nn.Conv2d(1, 1, kernel_size=2, bias=0, padding=3)
conv2d.weight.data = torch.ones((1, 1, 2, 2), requires_grad=True)
X = torch.ones((3, 3))
compute_conv2d(conv2d, X)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 2., 2., 1., 0., 0.],
        [0., 0., 2., 4., 4., 2., 0., 0.],
        [0., 0., 2., 4., 4., 2., 0., 0.],
        [0., 0., 1., 2., 2., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<ViewBackward0>)

In [122]:
X = torch.rand((8, 8))
conv2d = nn.LazyConv2d(1, kernel_size=(5,3), padding=(3,1))
compute_conv2d(conv2d, X)

tensor([[-0.1063, -0.0253,  0.0845,  0.0448, -0.0345,  0.1142,  0.1202,  0.0467],
        [-0.0383,  0.0902,  0.0545,  0.0745,  0.1452, -0.0237,  0.1262,  0.1986],
        [-0.0262,  0.1004,  0.0451,  0.0195,  0.1482,  0.1413,  0.1731,  0.3790],
        [-0.2000,  0.1423,  0.2661,  0.1258, -0.0061,  0.2357,  0.1614,  0.2021],
        [ 0.0604,  0.2778,  0.1180,  0.0945,  0.1374,  0.0146,  0.1736,  0.3904],
        [ 0.1010,  0.1191, -0.0297,  0.1442,  0.0747,  0.3457,  0.2106,  0.2159],
        [-0.0541,  0.3006,  0.1476,  0.1790,  0.1811,  0.1311,  0.1008,  0.1405],
        [ 0.2326,  0.1149,  0.0349,  0.0292, -0.0924,  0.1327,  0.2886,  0.1933],
        [ 0.0283, -0.1086,  0.0338,  0.0612,  0.0931, -0.0927, -0.0406,  0.0652],
        [-0.0483,  0.0680, -0.1084, -0.1071,  0.1008, -0.0878, -0.0391, -0.0840]],
       grad_fn=<ViewBackward0>)

In [123]:
conv2d = nn.LazyConv2d(1, kernel_size=3, padding=1, stride=2)
compute_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [124]:
conv2d = nn.LazyConv2d(1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
compute_conv2d(conv2d, X).shape

torch.Size([2, 2])

In [125]:
def myConv2d(X, K):
    h = K.shape[0]
    w = K.shape[1]
    Y = torch.zeros((X.shape[0]-h+1, X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            #print("i: ", i, ",j: ", j)
            #print("Y[i][j]: ", Y[i][j])
            Y[i][j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

X=torch.ones((3,3))
K=torch.ones((2,2))
myConv2d(X,K).sum()

tensor(16.)

In [126]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [127]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [128]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [129]:
K.shape

torch.Size([2, 2, 2])

In [130]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [131]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [132]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [133]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6